# Continued Pretraining: Teaching New Language to an LLM
In this notebook, we'll use **Unsloth** to **continue pretraining** a 4-bit quantized model on a **new language** dataset.
We'll use Korean Wikipedia as an example!

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf==3.20.3 datasets huggingface_hub hf_transfer tyro
    !pip install --no-deps unsloth

# Load Pretrained Base Model
We load Mistral-7B v0.3 model in 4-bit quantized format using Unsloth.

In [2]:
from unsloth import FastLanguageModel
import torch

seq_length = 2048
data_precision = None
quantize_4bit = True

model_base, tokenizer_base = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length = seq_length,
    dtype = data_precision,
    load_in_4bit = quantize_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

# Add LoRA and Embedding Training
We inject LoRA layers and allow tuning of `embed_tokens` and `lm_head` for true continual pretraining.

In [3]:
model_base = FastLanguageModel.get_peft_model(
    model_base,
    r = 128,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
        "embed_tokens", "lm_head"
    ],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


# Prepare Korean Wikipedia Dataset
We'll load and format Korean Wikipedia dump to teach new language context.

In [4]:
from datasets import load_dataset

# custom korean wikipedia prompt
wiki_prompt_kr = """위키피디아 문서
### 제목: {}
### 본문:
{}"""

eos_token = tokenizer_base.eos_token

def prepare_wiki(examples):
    titles = examples["title"]
    articles = examples["text"]
    return {
        "text": [
            wiki_prompt_kr.format(t, a) + eos_token
            for t, a in zip(titles, articles)
        ]
    }

# load korean wikipedia 1% subset
wiki_data = load_dataset("wikimedia/wikipedia", "20231101.ko", split="train")
wiki_data = wiki_data.train_test_split(train_size=0.01)["train"]
wiki_data = wiki_data.map(prepare_wiki, batched=True)

README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/400M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/205M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/177M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/647897 [00:00<?, ? examples/s]

Map:   0%|          | 0/6478 [00:00<?, ? examples/s]

# Continued Pretraining on New Language
We pretrain model on Korean Wikipedia using `UnslothTrainer`.

In [5]:
from unsloth import UnslothTrainer, UnslothTrainingArguments, is_bfloat16_supported
from transformers import TrainingArguments

trainer_wiki = UnslothTrainer(
    model = model_base,
    tokenizer = tokenizer_base,
    train_dataset = wiki_data,
    dataset_text_field = "text",
    max_seq_length = seq_length,
    dataset_num_proc = 2,
    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,
        max_steps = 120,
        warmup_steps = 10,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        output_dir = "wiki_pretrain_outputs",
        logging_steps = 1,
        seed = 42,
        report_to = "none",
    )
)

trainer_wiki.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/6478 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,478 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 603,979,776/7,000,000,000 (8.63% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.575700
2,1.549000
3,1.632500
4,1.540500
5,1.411600
6,1.401700
7,1.466900
8,1.371400
9,1.334300
10,1.361000


TrainOutput(global_step=120, training_loss=1.3236946652332942, metrics={'train_runtime': 715.0801, 'train_samples_per_second': 2.685, 'train_steps_per_second': 0.168, 'total_flos': 8.930542996861747e+16, 'train_loss': 1.3236946652332942})

# Load Korean Alpaca-GPT4 Dataset
Instruction finetune after pretraining using Korean translated instructions.

In [6]:
korean_alpaca = load_dataset("FreedomIntelligence/alpaca-gpt4-korean", split="train")

alpaca_prompt_kr = """다음은 작업을 설명하는 명령입니다. 요청을 적절하게 완료하는 응답을 작성하세요.

### 지침:
{}

### 응답:
{}"""

def prepare_alpaca(examples):
    conversations = examples["conversations"]
    return {
        "text": [
            alpaca_prompt_kr.format(conv[0]["value"], conv[1]["value"]) + eos_token
            for conv in conversations
        ]
    }

korean_alpaca = korean_alpaca.map(prepare_alpaca, batched=True)

README.md:   0%|          | 0.00/124 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


alpaca-gpt4-korean.json:   0%|          | 0.00/51.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49969 [00:00<?, ? examples/s]

Map:   0%|          | 0/49969 [00:00<?, ? examples/s]

# Instruction Finetuning (Chat-style Korean Instructions)
We now finetune using `UnslothTrainer` again.

In [7]:
trainer_alpaca = UnslothTrainer(
    model = model_base,
    tokenizer = tokenizer_base,
    train_dataset = korean_alpaca,
    dataset_text_field = "text",
    max_seq_length = seq_length,
    dataset_num_proc = 2,
    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,
        max_steps = 120,
        warmup_steps = 10,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        output_dir = "instruction_finetune_outputs",
        logging_steps = 1,
        seed = 42,
        report_to = "none",
    )
)

trainer_alpaca.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/49969 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 49,969 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 603,979,776/7,000,000,000 (8.63% trained)


Step,Training Loss
1,1.146200
2,1.108600
3,1.118900
4,1.043300
5,0.947000
6,0.941500
7,0.748000
8,0.910300
9,0.819700
10,0.832700


TrainOutput(global_step=120, training_loss=0.8172337790330251, metrics={'train_runtime': 520.3645, 'train_samples_per_second': 3.69, 'train_steps_per_second': 0.231, 'total_flos': 5.579555460135322e+16, 'train_loss': 0.8172337790330251})

# Inference
Let's generate a sample Korean response after training.

In [8]:
FastLanguageModel.for_inference(model_base)

inputs = tokenizer_base(
    [
        alpaca_prompt_kr.format(
            "한국 전통음악의 특징을 설명하세요.", # explaining Korean traditional music
            ""
        )
    ],
    return_tensors="pt"
).to("cuda")

output_text = model_base.generate(**inputs, max_new_tokens=128, use_cache=True)
print(tokenizer_base.decode(output_text[0], skip_special_tokens=True))

다음은 작업을 설명하는 명령입니다. 요청을 적절하게 완료하는 응답을 작성하세요.

### 지침:
한국 전통음악의 특징을 설명하세요.

### 응답:
한국 전통음악은 다양한 장르와 스타일로 구성되어 있으며, 다양한 지역과 문화에 따라 다양한 특징을 가지고 있습니다. 한국 전통음악의 특징 중 하나는 다양한 범주의 음악 장르로 구성되어 있는 것입니다. 이 중 


# Save Final Finetuned Model
Save LoRA adapters and tokenizer locally.

In [9]:
model_base.save_pretrained("final_korean_lora_model")
tokenizer_base.save_pretrained("final_korean_lora_model")

('final_korean_lora_model/tokenizer_config.json',
 'final_korean_lora_model/special_tokens_map.json',
 'final_korean_lora_model/tokenizer.model',
 'final_korean_lora_model/added_tokens.json',
 'final_korean_lora_model/tokenizer.json')